<a href="https://colab.research.google.com/github/uygararas/ie423/blob/main/tasks/task8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
# Load datasets
jokes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/filter/JokeText.csv', header=None, names=['JokeText'])
ratings1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/filter/UserRatings1.csv', header=None)
ratings2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/filter/UserRatings2.csv', header=None)

# Combine the ratings datasets
ratings = pd.concat([ratings1, ratings2], axis=0).reset_index(drop=True)

# Ensure the ratings dataframe contains only numeric data
ratings = ratings.apply(pd.to_numeric, errors='coerce')

# Replace missing values with the mean rating (column-wise)
ratings_filled = ratings.apply(lambda col: col.fillna(col.mean()), axis=0)

# Ensure there are no missing values
print(f"Total missing values after filling: {ratings_filled.isnull().sum().sum()}")

In [3]:

# Split the data into training and testing sets
train_data, test_data = train_test_split(ratings_filled, test_size=0.2, random_state=42)
train_data = train_data.reset_index(drop=True)  # Reset the index
# Compute the cosine similarity matrix for the training data
cosine_sim = cosine_similarity(train_data)

# Print shape of the cosine similarity matrix
print(f"Cosine similarity matrix shape: {cosine_sim.shape}")
print("Shape of train_data:", train_data.shape)

Cosine similarity matrix shape: (161, 161)
Shape of train_data: (161, 36712)


In [4]:
# Function to recommend jokes
def recommend_jokes(user_id, num_jokes=5):
    # Ensure the user_id is within the valid range
    if user_id < 0 or user_id >= train_data.shape[0]:
        raise IndexError("User ID is out of range")

    # Get the similarity scores for the given user
    user_similarities = cosine_sim[user_id]

    # Get the indices of the most similar users
    similar_user_indices = user_similarities.argsort()[::-1][:num_jokes]
    similar_users_ratings = train_data.loc[similar_user_indices]

    # Aggregate ratings from similar users
    joke_scores = similar_users_ratings.mean(axis=0)
    joke_scores = joke_scores.sort_values(ascending=False)
    # Get the indices of the top-rated jokes **within the 'jokes' DataFrame**
    top_joke_indices = joke_scores[::-1][:num_jokes]
    # Filter indices to be within the valid range for the 'jokes' DataFrame
    valid_top_joke_indices = [i for i in top_joke_indices if i < jokes.shape[0]]

    # Get the top-rated jokes
    top_jokes = jokes.iloc[valid_top_joke_indices]['JokeText'].tolist()
    return top_jokes

In [5]:
# prompt: call the function with userid 0 and num jokes 5 and then print the result

recommendations = recommend_jokes(0, 5)
for recommendation in recommendations:
    print(recommendation)


Reaching the end of a job interview, the human resources person asked a
young engineer fresh out of Stanford,

"And what starting salary were you looking for?"

The engineer said, "In the neighborhood of $125,000 a year, depending
on the benefits package."

The interviewer said, "Well, what would you say to a package of 5-weeks 
vacation, 14 paid holidays, full medical and dental, company matching 
retirement fund to 50% of salary, and a company car leased every 2 years - 
say, a red Corvette?"

The Engineer sat up straight and said, "Wow! Are you kidding?"

And the interviewer replied, "Yeah, but you started it."

Reaching the end of a job interview, the human resources person asked a
young engineer fresh out of Stanford,

"And what starting salary were you looking for?"

The engineer said, "In the neighborhood of $125,000 a year, depending
on the benefits package."

The interviewer said, "Well, what would you say to a package of 5-weeks 
vacation, 14 paid holidays, full medical and d